# Even / Odd training empiar10311
N2N (Noise2Noise) denoising del volumen empiar10311. Utiliza 2 volúmenes $V_1$ y $V_2$, donde $V_1$ es el volumen formado por los slices Z pares "even" y $V_2$ es el volumen formado por los slices Z impares "odd".

In [ ]:
# %pip install tensorflow==2.4

## Creamos $V_1$ y $V_2$

In [ ]:
%run split_even_odd.ipynb

## Configuramos cryoCARE

In [ ]:
%%writefile train_data_config__evenodd.json
{
    "even": ["even.mrc"],
    "odd": ["odd.mrc"],
    "mask": [""],
    "patch_shape": [16, 16, 16],
    "num_slices": 360,
    "split": 0.9,
    "tilt_axis": "Y",
    "n_normalization_samples": 200,
    "path": "./even_odd_data",
    "overwrite": "True"  
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar10311
source ~/envs/cryoCARE/bin/activate
cryoCARE_extract_train_data.py --conf train_data_config__evenodd.json

## Entrenamos

In [ ]:
%%writefile train_config__evenodd.json
{
  "train_data": "./even_odd_data",
  "epochs": 50,
  "steps_per_epoch": 200,
  "batch_size": 16,
  "unet_kern_size": 3,
  "unet_n_depth": 3,
  "unet_n_first": 16,
  "learning_rate": 0.0004,
  "model_name": "model",
  "path": "./",
  "gpu_id": [1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar10311
source ~/envs/cryoCARE/bin/activate
cryoCARE_train.py --conf train_config__evenodd.json

In [ ]:
service = DriveHandler.get_drive_service()
handler = DriveHandler.MrcDriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'
uploaded_file_id = handler.upload_mrc(
    local_file_path='model.tar.gz',
    drive_file_name='empiar10311_stack_crop_denoise__cryoCARE_even_odd__model.tar.gz',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

## Inferimos

In [ ]:
%%bash
ln -sf /nas/vruiz/vols/empiar10311_stack_crop.mrc .
#cd even_odd_denoised
#ln -sf /nas/vruiz/vols/empiar10311_stack_crop.mrc .

In [ ]:
%%writefile predict_config__evenodd.json
{
    "path": "./model.tar.gz",
    "even": ["empiar10311_stack_crop.mrc"], 
    "odd": ["empiar10311_stack_crop.mrc"],
    "n_tiles": [1,1,1],
    "output": "even_odd_denoised",
    "overwrite": "True",
    "gpu_id": [1]
}

In [ ]:
%%bash
#cd /nas/vruiz/cryoCARE/empiar10311
pwd
source ~/envs/cryoCARE/bin/activate
cryoCARE_predict.py --conf predict_config__evenodd.json || true

In [ ]:
from my_google_auth import DriveHandler

In [ ]:
service = DriveHandler.get_drive_service()
handler = DriveHandler.MrcDriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'
uploaded_file_id = handler.upload_mrc(
    local_file_path='even_odd_denoised/empiar10311_stack_crop.mrc',
    drive_file_name='empiar10311_stack_crop_denoise__cryoCARE_even_odd.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
import mrcfile
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
def read_MRC(path_file):
    return mrcfile.read(mrc_file_path)

In [ ]:
mrc_file_path = '/nas/vruiz/vols/empiar10311_stack_crop.mrc'
original_volume = read_MRC(mrc_file_path)

In [ ]:
original_volume.shape

In [ ]:
mrc_file_path = 'even_odd_denoised/empiar10311_stack_crop.mrc'
denoised_volume = read_MRC(mrc_file_path)

In [ ]:
denoised_volume.shape

In [ ]:
# Choose a slice index in the middle of the volume for a good comparison
slice_idx = original_volume.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(original_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised_volume[slice_idx, :, :].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N Even/Odd Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised_volume.shape[0]//2
plt.imshow(denoised_volume[slice_idx, 200:600, 200:600], cmap="gray")
plt.savefig("denoise_even_odd.pdf")

In [ ]:
service = DriveHandler.get_drive_service()
handler = DriveHandler.MrcDriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'
uploaded_file_id = handler.upload_mrc(
    local_file_path='denoise_even_odd.pdf',
    drive_file_name='empiar10311_stack_crop_denoise__cryoCARE_even_odd.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
%pip install "self_fourier_shell_correlation @ git+https://github.com/vicente-gonzalez-ruiz/self_fourier_shell_correlation"

In [ ]:
%pip show self_fourier_shell_correlation

In [ ]:
import sys
print(sys.executable)

In [ ]:
%pip install "shuffling @ git+https://github.com/vicente-gonzalez-ruiz/shuffling"

In [ ]:
%pip install opencv-python

In [ ]:
%pip install "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
from self_fourier_shell_correlation import fsc_utils as fsc

In [ ]:
import mrcfile

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
even = mrcfile.read("even.mrc")

In [ ]:
odd = mrcfile.read("odd.mrc")

In [ ]:
spatial_freq, fsc_values = fsc.fourier_shell_correlation(even, odd, shell_thickness=0.01)

In [ ]:
fsc.plot_fsc(spatial_freq, fsc_values, "Spatial Frequency (1/voxel units)", "FSC", "FSC(even, odd)")